In [4]:
import numpy as np
import pandas as pd
import csv
import nltk
import spacy
import re,string, unicodedata
import emoji
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import emoji_data_python
from collections import Counter
from textblob import TextBlob
from textblob import Word
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from spellchecker import SpellChecker
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from plotly.subplots import make_subplots
from bs4 import BeautifulSoup
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Inteli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
base_og = pd.read_csv('BaseDadosBTG2.csv')
base_og.head()

,id,"""dataPublicada""","""autor""","""texto""","""sentimento""","""tipoInteracao""","""anomalia""","""probabilidadeAnomalia""","""linkPost""","""processado""","""contemHyperlink"""
0,1,"""2022-03-04 09:38:00""",v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,marcação,1,52,https://www.instagram.com/p/CarqO6drRux/,0,0
1,2,"""2022-03-04 09:29:00""",winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,marcação,0,43,https://www.instagram.com/p/CarpQUluh7z/,0,0
2,3,"""2022-03-04 09:20:00""",marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,marcação,0,45,https://www.instagram.com/p/CaroK9Nu8tC/,0,0
3,4,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0
4,5,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0


In [9]:
base_lemmatizada = pd.read_csv('lematizacao.csv')
base_lemmatizada.head()

,Unnamed: 0,autor,texto,sentimento,tokens
0,0,v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,"['confira', 'o', 'resultado', 'de o', 'nosso',..."
1,1,winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,"['Alvarez', 'Marsal', 'estara', 'conosco', 'sp..."
2,2,marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,"['repost', 'btg', 'With', 'makerepost', 'enten..."
3,3,lmviapiana,Minuto touro de ouro,POSITIVE,"['minuto', 'touro', 'ouro']"
4,4,vanilson_dos,@ricktolledo Sim,NEUTRAL,"['Ricktolledo', 'sim']"


In [11]:
apenas_tokens = base_lemmatizada['tokens']
apenas_tokens

0        ['confira', 'o', 'resultado', 'de o', 'nosso',...
1        ['Alvarez', 'Marsal', 'estara', 'conosco', 'sp...
2        ['repost', 'btg', 'With', 'makerepost', 'enten...
3                              ['minuto', 'touro', 'ouro']
4                                   ['Ricktolledo', 'sim']
                               ...                        
12188    ['um', 'noite', 'encontro', 'muito', 'conhecim...
12189    ['erro', 'financeiro', 'eliminar', 'antes', 'd...
12190    ['estar', 'muito', 'grato', 'todo', 'esforco',...
12191    ['dorsodamaocomdedoindicadorapontandoparaadire...
12192    ['btg', 'Morning', 'call', 'nao', 'este', 'mai...
Name: tokens, Length: 12193, dtype: object

In [17]:
import gensim
from gensim.models import Word2Vec


# sentences = [
#     ['I', 'love', 'natural', 'language', 'processing'],
#     ['Word', 'embeddings', 'are', 'useful', 'for', 'NLP'],
#     ['AI', 'and', 'machine', 'learning', 'are', 'exciting', 'fields'],
#     ['Python', 'is', 'a', 'popular', 'programming', 'language'],
#     ['I', 'enjoy', 'working', 'with', 'data'],
#     ['teste', 'do teste', 'do teste'],
# ]


# Training the Word2Vec model
model = Word2Vec(apenas_tokens, vector_size=100, window=5, min_count=1, workers=4)

# Testing the Word2Vec model
word = [4]
word_vector = model.wv[word]
similar_words = model.wv.most_similar(word)

# Displaying the results
print(f'Word vector for "{word}": {word_vector}')

Word vector for "[4]": [[-0.18440181 -0.70692027  0.14540437 -0.06715839 -0.29108542  0.29896435
   0.29018244 -0.06582729 -0.36412653  0.08705666  0.17910653  0.6456244
   0.00434769  0.17826137  0.17758249 -0.46255916  0.13421197  0.6120299
  -0.23929645  0.18897079  0.5568706   0.37082025 -0.0102634   0.6399023
  -0.38245815  0.10504126 -0.2178039   0.21783876 -0.3223735   0.11806782
  -0.6546671   0.28021196  0.14969583 -0.55453485  0.36743128  0.28195968
  -0.27520126 -0.6376135  -0.05573766  0.13165519 -0.19031593 -0.6708178
  -0.4570169   0.7427955   0.7743932   0.35087773 -0.78884083  0.04699248
   1.432523   -0.38823816 -0.24505648 -0.06951141  0.84842014 -0.13937482
  -0.30240613  0.84466904 -0.21913762 -0.45585352  0.35628298  0.31645387
   0.72172534 -0.2727523   0.19423352 -0.09426349  0.27470103  0.12017887
  -0.41622162  0.61761    -0.03961831  0.25175858  0.08553329  0.45359612
  -0.28205496 -0.5278978   0.2252326   0.16085958  0.1269081   0.11290976
   0.36895093 -0.15

In [18]:
print(f'Similar words to "{word}": {similar_words}')

Similar words to "[4]": [('o', 0.7615816593170166), ('i', 0.7035526037216187), (' ', 0.6459622383117676), (',', 0.6372575163841248), ('a', 0.5918338298797607), ('s', 0.5772249102592468), ('v', 0.5505048632621765), ("'", 0.5356466174125671), ('r', 0.5106390714645386), ('t', 0.4894501566886902)]
